# Utilising Detectron2 to perform fast automated structural analysis for seismic sections

## Revolutionizes Seismic Image Interpretation with Advanced Deep Learning, using CNNs 

<img src="https://i.imgur.com/wY8BOfV.jpg" width="500">

Welcome to Automatic seismic Interpretation using Deep Learning!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Detectron2

The !nvidia-smi command is used to query and display information about the NVIDIA GPU(s) available on your system. To utilize this command in a Jupyter Notebook, simply include ! before the command, like this:

In [ ]:
!nvidia-smi

When executed, this command will output real-time details about the GPU(s), including usage statistics, memory information, temperature, and more. This information is valuable for monitoring GPU performance and can be beneficial when optimizing code for GPU acceleration.

In [ ]:
%cd /content/drive/MyDrive/detecron2

The `%cd /content/drive/MyDrive/detecron2` command in Jupyter Notebooks is utilized to set the current working directory to "/content/drive/MyDrive/detecron2". Adjust this path according to the location of your Detectron2 project. Ensuring the correct working directory is crucial for seamless code execution and resource access within the project.

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

## Import Libraries

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

## Setup Detectron2

This code snippet sets up the environment for using Detectron2, a popular computer vision library for object detection. It includes the following steps:

Setting up Detectron2 Logger: Initializes the logger for Detectron2 to handle logging.
Importing Common Libraries: Imports essential libraries like NumPy, OpenCV (cv2), and tools for displaying images in Google Colab.
Importing Detectron2 Utilities: Imports necessary modules from Detectron2, including model zoo, predictor, configuration, visualizer, and data utilities.
This code is a foundational setup for subsequent tasks involving Detectron2, such as configuring models, loading datasets, and performing object detection.

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Load Dataset

In [ ]:
# !unzip data.zip

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/detecron2

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("customtrain", {}, "./trainval.json", "./images")

In [ ]:
sample_metadata = MetadataCatalog.get("customtrain")
dataset_dicts = DatasetCatalog.get("customtrain")

## Visualise the Dataset

1. `import random`: Imports the `random` module for generating random samples.

2. `for d in random.sample(dataset_dicts, 5):`:
   - Iterates over five random samples from `dataset_dicts`, assuming it's a list of dictionaries representing a dataset.

3. `img = cv2.imread(d["file_name"])`:
   - Reads the image file specified by the "file_name" key in the current dataset dictionary (`d`) using OpenCV (`cv2`).

4. `visualizer = Visualizer(img[:, :, ::-1], metadata=sample_metadata, scale=0.5)`:
   - Creates a `Visualizer` object to visualize the image.
   - The `metadata` parameter is assumed to contain metadata for visualization.
   - The `scale` parameter sets the scale of the visualization to 0.5.

5. `vis = visualizer.draw_dataset_dict(d)`:
   - Uses the `Visualizer` to draw bounding boxes and annotations on the image based on the information in the current dataset dictionary (`d`).

6. `cv2_imshow(vis.get_image()[:, :, ::-1])`:
   - Displays the visualized image using `cv2_imshow`, which is a Google Colab-specific function for showing images.
   - The `[:, :, ::-1]` part reverses the order of color channels to match the expected format for image display.

Overall, the code randomly selects five samples from a dataset (`dataset_dicts`), reads and visualizes the corresponding images, including any annotations or bounding boxes, and displays them using the `cv2_imshow` function in a Google Colab environment.

In [ ]:
import random

for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=sample_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

## Configuration for training a Mask R-CNN model

1. `from detectron2.engine import DefaultTrainer`: Imports the `DefaultTrainer` class from the Detectron2 engine module.

2. `cfg = get_cfg()`: Initializes a Detectron2 configuration (`cfg`) using `get_cfg()`.

3. `cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))`:
   - Merges the configuration settings from the COCO instance segmentation model configuration file (`mask_rcnn_R_50_FPN_3x.yaml`) into the current configuration.

4. `cfg.DATASETS.TRAIN = ("customtrain",)`: Specifies the training dataset, assumed to be named "customtrain".

5. `cfg.DATASETS.TEST = ()`: Specifies that no separate test dataset is used.

6. `cfg.DATALOADER.NUM_WORKERS = 2`: Sets the number of data loader workers to 2 for parallel data loading.

7. `cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")`:
   - Initializes the model weights from the COCO instance segmentation model available in the model zoo.

8. `cfg.SOLVER.IMS_PER_BATCH = 8`: Sets the number of images per batch during training to 8.

9. `cfg.SOLVER.BASE_LR = 0.0025`: Sets the base learning rate for the optimizer.

10. `cfg.SOLVER.MAX_ITER = 3200`: Sets the maximum number of training iterations to 3200.

11. `cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 640`: Sets the batch size per image for region of interest (ROI) heads to 640.

12. `cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3`: Specifies the number of classes in the dataset, assumed to be 3.

13. `os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)`: Creates the output directory specified in the configuration if it does not exist.

14. `trainer = DefaultTrainer(cfg)`: Initializes a `DefaultTrainer` object with the provided configuration.

15. `trainer.resume_or_load(resume=False)`: Resumes training from a checkpoint if available or loads the pre-trained weights.

16. `trainer.train()`: Initiates the training process using the specified configuration.

In summary, the code sets up a configuration for training a Mask R-CNN model on a custom dataset, configures various parameters such as batch size and learning rate, and then starts the training process using the `DefaultTrainer` class.

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("customtrain",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8 # 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 3200 # 1200    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 640 # 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Configuring Model Inference for Custom Dataset

1. **Setting Model Weights Path:**
   - `cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")`: Specifies the path to the final trained model weights generated during the training process. This ensures that the inference uses the latest model state.

2. **Setting Inference Threshold:**
   - `cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5`: Establishes a threshold for object detection during testing. Objects with confidence scores below this threshold are not considered during inference.

3. **Setting Test Dataset:**
   - `cfg.DATASETS.TEST = ("customtrain", )`: Defines the dataset (assumed to be named "customtrain") to be used for model testing. This ensures that the model is evaluated on the specified dataset during inference.

4. **Initializing Predictor:**
   - `predictor = DefaultPredictor(cfg)`: Creates an instance of the `DefaultPredictor` class with the configured settings. The `DefaultPredictor` is a convenient wrapper that simplifies the process of making predictions using a trained Detectron2 model.

In summary, this code segment prepares the Mask R-CNN model for inference on a custom dataset by specifying the model weights, setting a confidence threshold for object detection, defining the test dataset, and initializing a predictor for making predictions.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("customtrain", )
predictor = DefaultPredictor(cfg)

##  Visualizing Model Predictions on Random Samples

The provided code snippet is responsible for visualizing the predictions made by a Mask R-CNN model on a few randomly selected samples from a dataset. Below is an explanation of the key components:

1. **Model Prediction:**
   - `outputs = predictor(im)`: Uses the previously configured predictor to obtain model predictions on the input image (`im`).

2. **Visualization Settings:**
   - `v = Visualizer(im[:, :, ::-1], metadata=sample_metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)`: Initializes a visualizer (`v`) with the input image, metadata, and visualization settings. The `ColorMode.IMAGE_BW` parameter is set to remove colors from unsegmented pixels.

3. **Drawing Predictions:**
   - `v = v.draw_instance_predictions(outputs["instances"].to("cpu"))`: Draws the instance predictions (bounding boxes, masks, etc.) on the visualizer. The predictions are obtained from the model's output and converted to CPU format for visualization.

4. **Displaying Visualization:**
   - `cv2_imshow(v.get_image()[:, :, ::-1])`: Displays the final visualized image using OpenCV. The image is converted to the RGB format for proper display.

In summary, this code segment generates visualizations for model predictions on a few randomly selected images from the dataset. The visualizations include bounding boxes, masks, and other instance-level information predicted by the Mask R-CNN model.

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 4):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=sample_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

## Visualizing Model Predictions on a Single Image

1. **Read Image:**
   - `im = cv2.imread("/content/drive/MyDrive/detecron2/images/acoustic_impedance__Section_5_Ems.png")`: Reads the specified image file using OpenCV and stores it in the variable `im`.

2. **Model Prediction:**
   - `outputs = predictor(im)`: Utilizes the predictor to obtain model predictions on the input image (`im`).

3. **Visualization Settings:**
   - `v = Visualizer(im[:, :, ::-1], metadata=sample_metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)`: Initializes a visualizer (`v`) with the input image, metadata, and visualization settings. The `ColorMode.IMAGE_BW` parameter is set to remove colors from unsegmented pixels.

4. **Drawing Predictions:**
   - `v = v.draw_instance_predictions(outputs["instances"].to("cpu"))`: Draws the instance predictions (bounding boxes, masks, etc.) on the visualizer. The predictions are obtained from the model's output and converted to CPU format for visualization.

5. **Displaying Visualization:**
   - `cv2_imshow(v.get_image()[:, :, ::-1])`: Displays the final visualized image using OpenCV. The image is converted to the RGB format for proper display.

In summary, this code segment visualizes the predictions made by a Mask R-CNN model on a specific image, showing bounding boxes, masks, and other instance-level details.

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/images/acoustic_impedance__Section_5_Ems.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/images/acoustic_impedance__Section_8_Ems.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/images/acoustic_impedance__Section_9_Ems.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/images/acoustic_impedance__Section_3_Ems.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

## Testing from the Coalmine cross-sections

## Gathering Image File Paths from a Directory

1. **Import Modules:**
   - `import os`: Imports the Python module for interacting with the operating system.
   - `from os import listdir`: Imports the `listdir` function from the `os` module.

2. **Specify Directory and Initialize List:**
   - `folder_dir = '/content/drive/MyDrive/detecron2/testing-crosssections-images'`: Specifies the directory path where the images are located.
   - `images_ls = []`: Initializes an empty list (`images_ls`) to store the file paths of image files.

3. **Iterate Through Files in the Directory:**
   - `for images in os.listdir(folder_dir):`: Iterates through the files in the specified directory.
   - `if (images.endswith(".png") or images.endswith(".jpg") or images.endswith(".jpeg")):`: Checks if the file has one of the specified image file extensions (`.png`, `.jpg`, or `.jpeg`).
   - `images_ls.append(folder_dir + "/" +  images)`: If the file is an image, appends its complete path to the `images_ls` list.

In summary, this code segment creates a list (`images_ls`) containing the full file paths of image files with specific extensions (`png`, `jpg`, and `jpeg`) found within a specified directory (`folder_dir`).

In [ ]:
# import the modules
import os
from os import listdir

# get the path or directory
folder_dir = '/content/drive/MyDrive/detecron2/testing-crosssections-images'
images_ls = []
for images in os.listdir(folder_dir):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        images_ls.append(folder_dir + "/" +  images)

## Looping Through Images and Making Predictions

1. **Loop Over Image Paths:**
   - `for image_file in images_ls:`: Initiates a loop to iterate over each image file path in the `images_ls` list.

2. **Construct Image Path:**
   - `image_path = os.path.join(folder_dir, image_file)`: Constructs the full path of the current image by joining the specified directory (`folder_dir`) and the image file name.

3. **Read Image and Make Predictions:**
   - `im = cv2.imread(image_path)`: Reads the image using OpenCV.
   - `outputs = predictor(im)`: Uses the pre-trained model (`predictor`) to make predictions on the current image.

4. **Visualize Predictions:**
   - `v = Visualizer(im[:, :, ::-1], metadata=sample_metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)`: Creates a visualizer object for the current image.
   - `v = v.draw_instance_predictions(outputs["instances"].to("cpu"))`: Draws the predicted instances on the image using the visualizer.

5. **Display Image with Predictions:**
   - `cv2_imshow(v.get_image()[:, :, ::-1])`: Displays the image with the overlaid predicted instances.

This code segment essentially automates the process of loading multiple images, making predictions, and visualizing the results, allowing for a quick inspection of the model's performance on a set of test images.

In [ ]:
# Loop over the images and predict each one
for image_file in images_ls:
    image_path = os.path.join(folder_dir, image_file)
    im = cv2.imread(image_path)
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1],
                   metadata=sample_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2_imshow(v.get_image()[:, :, ::-1])

## Testing

## List Image Files in a Directory

The provided code snippet aims to create a list (`images_ls`) containing the file paths of image files (with extensions `.png`, `.jpg`, or `.jpeg`) located in a specified directory (`folder_dir`).

1. **Import Necessary Modules:**
   - `import os`: Imports the `os` module for operating system-related functions.
   - `from os import listdir`: Imports the `listdir` function from the `os` module.

2. **Define Directory and Initialize List:**
   - `folder_dir = '/content/drive/MyDrive/detecron2/test-images'`: Specifies the directory path where the images are located.
   - `images_ls = []`: Initializes an empty list (`images_ls`) to store the file paths of eligible image files.

3. **Loop Over Files in the Directory:**
   - `for images in os.listdir(folder_dir):`: Initiates a loop to iterate over each file in the specified directory.

4. **Filter Image Files:**
   - The `if` condition checks whether the file has an extension of `.png`, `.jpg`, or `.jpeg`.
   - `images_ls.append(folder_dir + "/" + images)`: Appends the full file path to the `images_ls` list for eligible image files.

After running this code snippet, the list `images_ls` will contain the full paths of image files in the specified directory, filtered based on the mentioned file extensions.

In [ ]:
# import the modules
import os
from os import listdir

# get the path or directory
folder_dir = '/content/drive/MyDrive/detecron2/test-images'
images_ls = []
for images in os.listdir(folder_dir):
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        images_ls.append(folder_dir + "/" +  images)

## Print Image File Paths and Read Images

The provided code snippet iterates over the list of image file paths (`images_ls`) and prints each file path. Additionally, there is a commented line to demonstrate reading and printing images using OpenCV.

**Code Explanation:**
```python
# Loop Over Image File Paths
for i in range(len(images_ls)):
    # Print the Image File Path
    print(images_ls[i])
    
    # Uncomment the line below to Read and Print Images Using OpenCV
    # image = cv2.imread(images_ls[i])
    # print(image)
```

- **Loop Over Image File Paths:**
  - `for i in range(len(images_ls)):`
  - Initiates a loop to iterate over each image file path in the `images_ls` list.

- **Print Image File Paths:**
  - `print(images_ls[i])`
  - Prints the file path of each image.

- **Read and Print Images (Commented):**
  - The following two lines are commented (prefixed with `#`), but you can uncomment them to read and print images using OpenCV.
  - `image = cv2.imread(images_ls[i])`: Reads the image using OpenCV.
  - `print(image)`: Prints the image array.

**Note:**
- Uncomment the lines if you want to read and print the images. This requires the OpenCV library (`cv2`).
- The code is currently set to only print the file paths. If you uncomment and run the OpenCV-related lines, it will also read and print the images.

In [ ]:
for i in range(len(images_ls)):
  print(images_ls[i])
  # print(cv2.imread(images_ls[i]))

In [ ]:
folder_path = r"/content/drive/MyDrive/detecron2/test-images"

## Predict and Visualize Images Using the Trained Model

The provided code snippet predicts and visualizes images using a trained model. It loops over a list of image file paths (`images_ls`), reads each image using OpenCV, and then uses the trained model (`predictor`) to make predictions. The predictions are visualized using the Detectron2 library's `Visualizer`, and the images with predicted instances are displayed.

**Code Explanation:**
```python
# Loop Over Image File Paths for Prediction and Visualization
for image_file in images_ls:
    # Construct the Full Image Path
    image_path = os.path.join(folder_path, image_file)
    
    # Read the Image Using OpenCV
    im = cv2.imread(image_path)
    
    # Make Predictions using the Trained Model
    outputs = predictor(im)

    # Visualize Predictions using the Detectron2 Visualizer
    v = Visualizer(im[:, :, ::-1],
                   metadata=sample_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Display the Visualized Image
    cv2_imshow(v.get_image()[:, :, ::-1])
```

- **Loop Over Image File Paths:**
  - `for image_file in images_ls:`
  - Initiates a loop to iterate over each image file path in the `images_ls` list.

- **Construct Full Image Path:**
  - `image_path = os.path.join(folder_path, image_file)`
  - Combines the folder path (`folder_path`) with the current image file path to create the full image path.

- **Read the Image Using OpenCV:**
  - `im = cv2.imread(image_path)`
  - Reads the image using OpenCV.

- **Make Predictions:**
  - `outputs = predictor(im)`
  - Uses the trained model (`predictor`) to make predictions on the current image.

- **Visualize Predictions Using Detectron2:**
  - `v = Visualizer(im[:, :, ::-1], metadata=sample_metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)`
  - Initializes the Visualizer with the image, metadata, and visualization settings.
  - `v = v.draw_instance_predictions(outputs["instances"].to("cpu"))`
  - Draws the predicted instances on the image using the Visualizer.

- **Display Visualized Image:**
  - `cv2_imshow(v.get_image()[:, :, ::-1])`
  - Displays the visualized image with predicted instances.

**Note:**
- Ensure that the necessary libraries (`detectron2`, `cv2`) are properly installed and imported before running the code.
- The code assumes that `folder_path` is defined and contains the images to be predicted and visualized.
- Adjustments to the code may be needed based on the specific requirements of your project.

In [ ]:
# Loop over the images and predict each one
for image_file in images_ls:
    image_path = os.path.join(folder_path, image_file)
    im = cv2.imread(image_path)
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1],
                   metadata=sample_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/fig-1.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/show_picture.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/verng02_1zoomclean1.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/200401-inline-comparisons-slider-full-integrity.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/Legacy_stack_before.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/some-machine-learning-applications-in-seismic-interpretation-hero.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/fig-1.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/geoscience_subsurface_imaging_ima_data_after.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
im = cv2.imread("/content/drive/MyDrive/detecron2/test-images/seismic_10042019-600x270-c-default.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

### Evaluation
We can evaluate its performance using AP metric implemented in COCO API.


## Evaluate Model Performance on Validation Dataset

This code snippet evaluates the performance of a trained model on a validation dataset using the COCOEvaluator from the Detectron2 library. It calculates metrics such as bounding box (bbox) and segmentation (segm) accuracy. The evaluation results are printed, providing insights into the model's performance on the specified validation dataset.

**Code Explanation:**
```python
# Import Necessary Modules
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Create COCO Evaluator
evaluator = COCOEvaluator("customtrain", ("bbox", "segm"), False, output_dir="./output/")

# Build Detection Test Loader for Validation Dataset
val_loader = build_detection_test_loader(cfg, "customtrain")

# Perform Inference on Validation Dataset and Print Results
print(inference_on_dataset(trainer.model, val_loader, evaluator))
```

- **Create COCO Evaluator:**
  - `evaluator = COCOEvaluator("customtrain", ("bbox", "segm"), False, output_dir="./output/")`
  - Initializes a COCOEvaluator for the "customtrain" dataset with evaluation metrics including bounding box (`bbox`) and segmentation (`segm`).

- **Build Detection Test Loader:**
  - `val_loader = build_detection_test_loader(cfg, "customtrain")`
  - Constructs a detection test loader for the validation dataset ("customtrain") using the specified configuration (`cfg`).

- **Perform Inference on Validation Dataset:**
  - `inference_on_dataset(trainer.model, val_loader, evaluator)`
  - Executes inference on the validation dataset using the trained model (`trainer.model`) and the created evaluator.
  - The results include metrics such as AP (average precision) for bounding box and segmentation tasks.

- **Print Evaluation Results:**
  - `print(...)`
  - Prints the evaluation results, providing insights into the model's performance on the specified validation dataset.

**Note:**
- Ensure that the necessary libraries (`detectron2`) are properly installed and imported before running the code.
- Adjustments to the code may be needed based on the specific configurations and datasets used in your project.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("customtrain", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "customtrain")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Getting the custom config file

In [ ]:
f = open('config.yml', 'w')
f.write(cfg.dump())
f.close()

### By: Ramy Abdallah 

### Thank you!!